# f) Ngonye Falls Power and Energy Periods
Produces summary, agrregated results from the daily power and energy modelling.


## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| [Model Name]_pe_daily.csv | Notebook: e_power_energy | Daily power output  |


## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|
|    [Model Name]_pe_daily.csv ||
|    [Model Name]_pe_monthly.csv||
|    [Model Name]_pe_yearly.csv||
|    [Model Name]_pe_percentiles.csv||
|    [Model Name]_pe_monthly_day.csv||
|    [Model Name]_pe_daily_slim.csv||
|    [Model Name]_pe_weekly.csv||
|    [Model Name]_pe_weekly_slim.csv||
|    [Model Name]_pe_calmonthly.csv||
|    [Model Name]_pe_fdc.csv||
|    [Model Name]_pe_power_exceed.csv||
|    models.csv| |

## Libraries and Setup

In [69]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math
import statsmodels.tsa.stattools as statools

## Parameters

In [70]:
#Papermill parameters
file_infix='_WC1_1' #'ngonye_daily'#'WC1_1'
folder='CC/'

In [71]:
input_data='./input_data/'
output_data='./output_data/'

In [72]:
model_name='Base Case'#MW153#MW144 'MW162'#'Fixed Tailwater'#'Base Case' Headpond50cm EWRAllCs
dryrun=False #Dont write output files

In [73]:
models=pd.read_csv(input_data + "models.csv").set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'
if out_prefix!='base':
    out_prefix="scenarios/" + out_prefix
    
plant_capacity=model['Capacity']#180
canal_capacity=plant_capacity/180*1100#1100

model

OutputPrefix                       base
Description                   Base Case
EWRCatSet              Recommendation 1
HeadpondLift                          0
Capacity                            180
FixedTailwater                        0
Mean                             832484
P95                              718504
P90                              756610
P75                              792216
P50                              838165
P25                              876248
P10                              921915
P5                               958253
StDev                           81971.2
CoefVar                       0.0984658
Max                         1.01897e+06
Min                              493939
CapFactor                      0.527958
CalYearMean                      832587
CalYearStDev                    78237.2
CalYearCoefVar                0.0939688
AutoCorrel                          NaN
Power_max                       193.908
CapFactor_Power_max             0.49015


## Load Daily Data

Load the daily time series which includes the exceedance values used for calculating daily EWRs.

In [74]:
daily=pd.read_csv(output_data + folder + 'ngonye_daily' + file_infix + '_base_pe.csv') #ngonye_daily_WC1_1_base_pe
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)
daily.head(5)

,Year,Day,Month,CC_Factor,SampleDate,SampleFlow,Impact_2070,Flow,EWRRefExceedance,EWRBandNo,...,EffGen2,EffGen3,EffGen4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec
Date,,,,,,,,,,,,,,,,,,,,,
2017-10-01,2017,1,10,0.3375,1933-10-01,188.719829,-0.16,178.528958,0.5,3,...,NaN,NaN,NaN,16.790064,NaN,NaN,NaN,16.790064,402.961538,0.094047
2017-10-02,2017,2,10,0.3375,1933-10-02,188.719829,-0.16,178.528958,0.5,3,...,NaN,NaN,NaN,16.790064,NaN,NaN,NaN,16.790064,402.961538,0.094047
2017-10-03,2017,3,10,0.3375,1933-10-03,188.719829,-0.16,178.528958,0.5,3,...,NaN,NaN,NaN,16.790064,NaN,NaN,NaN,16.790064,402.961538,0.094047
2017-10-04,2017,4,10,0.3375,1933-10-04,185.778706,-0.16,175.746655,0.5,3,...,NaN,NaN,NaN,16.216244,NaN,NaN,NaN,16.216244,389.189859,0.092271
2017-10-05,2017,5,10,0.3375,1933-10-05,185.778706,-0.16,175.746655,0.5,3,...,NaN,NaN,NaN,16.216244,NaN,NaN,NaN,16.216244,389.189859,0.092271


## Monthly


In [117]:
if not 'MonthId' in daily.columns:
    daily['MonthId']=daily['Year']+daily['Month']/100

if not 'WaterMonth' in daily.columns:
    daily['WaterMonth']=daily.apply((lambda x: (x['Month']-9 if x['Month']>=10 else x['Month']+3)),axis=1)

In [119]:

monthly=daily.groupby("MonthId").mean().drop(['Day','EWRBandNo','EWRRefExceedance','Energy'],axis=1)

if 'VicFalls' in monthly.columns:
    monthly=monthly.drop(['VicFalls','Conversion','Volume','WaterDay','WaterWeek','Volume'],axis=1)

monthly=monthly.join(daily.groupby("MonthId").agg(    
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),   
   Turbines_max=('Turbines', 'max'),    
   Turbines_min=('Turbines', 'min'),   
   Energy=('Energy', 'sum'),
))
monthly['Days']=monthly.apply(lambda x: pd.Period(year=x['Year'],month=x['Month'],freq='D').days_in_month,axis=1)
monthly['CapFactor']=monthly['Energy']/(plant_capacity*24*monthly['Days'])

monthly.tail(12)

,Year,Month,CC_Factor,SampleFlow,Impact_2070,Flow,EWRChannelA,EWRChannelC,EWRChannelD,EWRChannelE,...,WaterDay,WaterWeek,WaterMonth,Power_max,Power_min,Turbines_max,Turbines_min,Energy,Days,CapFactor
MonthId,,,,,,,,,,,,,,,,,,,,,
2052.10,2052.0,10.0,0.7750,305.412179,-0.16,267.541068,41.000000,6.000000,4.000000,3.000000,...,16.0,2.741935,1.0,42.383823,29.548406,1,1,25539.417260,31,0.190707
2052.11,2052.0,11.0,0.7750,356.388894,-0.20,301.148616,40.000000,6.000000,4.000000,3.000000,...,46.5,7.066667,2.0,60.304864,28.556457,2,1,30679.438284,30,0.236724
2052.12,2052.0,12.0,0.7750,553.985963,-0.17,480.998312,42.645161,7.322581,5.322581,3.322581,...,77.0,11.419355,3.0,95.045876,59.056827,2,2,56659.043820,31,0.423081
2053.01,2053.0,1.0,0.7875,790.184416,-0.18,678.175775,46.000000,7.000000,6.000000,4.000000,...,108.0,15.870968,4.0,137.973050,91.567529,3,2,81993.681745,31,0.612259
2053.02,2053.0,2.0,0.7875,1755.235893,-0.16,1534.076170,49.000000,8.000000,7.000000,5.000000,...,137.5,20.071429,5.0,190.684241,115.696476,4,3,105619.813553,28,0.873180
2053.03,2053.0,3.0,0.7875,6893.057191,-0.41,4667.461351,62.193548,12.064516,13.774194,15.838710,...,167.0,24.258065,6.0,128.321585,0.000000,4,0,73425.435795,31,0.548278
2053.04,2053.0,4.0,0.7875,7422.717087,-0.29,5727.554072,84.000000,14.000000,20.000000,27.000000,...,197.5,28.666667,7.0,108.583744,0.000000,4,0,25863.853941,30,0.199567
2053.05,2053.0,5.0,0.7875,3738.769494,-0.24,3032.142060,71.000000,14.000000,18.000000,24.000000,...,228.0,33.000000,8.0,163.412260,107.297588,4,4,102815.122128,31,0.767735
2053.06,2053.0,6.0,0.7875,1862.872723,-0.24,1510.789778,65.333333,12.666667,14.666667,16.666667,...,258.5,37.333333,9.0,187.379979,147.011765,4,3,124091.329704,30,0.957495


## Yearly

In [120]:
if not 'WaterYear' in daily.columns:
    daily['WaterYear']=daily.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)

In [121]:
yearly=daily.groupby("WaterYear").agg(
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
)
yearly['CapFactor']=yearly['Energy']/(plant_capacity*24*365)


In [122]:
yearly['Prop4Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==4,1,0))/365)
yearly['Prop3Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==3,1,0))/365)
yearly['Prop2Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==2,1,0))/365)
yearly['Prop1Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==1,1,0))/365)
yearly['Prop0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0))/365)
yearly['Days0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0)))


## Power and energy exceedance

In [123]:
power_exceed=pd.DataFrame(index=np.arange(0,1.05,0.05))
power_exceed.index=power_exceed.index.rename('Exceedance')
power_exceed['Power']=np.percentile(daily['Power'],(1-power_exceed.index)*100)
power_exceed.head(3)

,Power
Exceedance,
0.00,194.130914
0.05,180.807470
0.10,172.040032


In [124]:
edc=pd.DataFrame(index=np.arange(0,1.05,0.05))
edc['Energy']=np.percentile(yearly['Energy'],(1-edc.index)*100)

edc.head(3)

,Energy
0.00,970011.092095
0.05,890701.073199
0.10,882060.318077


In [125]:
yearly['Exceedance']=pd.merge_asof(yearly.reset_index().sort_values('Energy'),edc.reset_index().sort_values('Energy'),left_on='Energy',right_on='Energy').set_index('WaterYear')['index']

## Calendar years
Jan-Dec rather than Oct-Sept

In [126]:
min_year=daily['Year'].min()
max_year=daily['Year'].max()


In [127]:
calyearly=daily.groupby("Year").agg(  
   Energy=('Energy', 'sum'),
).drop([min_year,max_year],axis=0)
calyearly['CapFactor']=calyearly['Energy']/(180*24*365)
calyearly.head(5)

,Energy,CapFactor
Year,,
2018,784569.949739,0.497571
2019,819608.058315,0.519792
2020,786002.166807,0.498479
2021,837570.288088,0.531184
2022,804559.854865,0.510249


## Monthly Day
15th of every month

In [128]:
monthly_day=daily.loc[daily['Day']==15]

## Daily Slim
Slim has less columns and values are rounded so file size is reduced

In [129]:
daily_slim=daily.loc[:,
                     ['Year',
                      'Month',
                      'Day',
                      'WaterYear',
                      'Flow',
                      'EWRRefExceedance',
                      'EWRBandNo',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'Turbines',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',    
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion': 2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,    
                    })
daily_slim.tail(12)

,Year,Month,Day,WaterYear,Flow,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,EffTurbine1,EffTurbine2,EffTurbine3,EffTurbine4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy
Date,,,,,,,,,,,,,,,,,,,,,
2053-09-19,2053,9,19,2052,403,0.0,1,48,8,7,...,0.95,0.0,0.0,0.0,56.6,0.0,0.0,0.0,56.6,1357
2053-09-20,2053,9,20,2052,403,0.0,1,48,8,7,...,0.95,0.0,0.0,0.0,56.6,0.0,0.0,0.0,56.6,1357
2053-09-21,2053,9,21,2052,396,0.0,1,48,8,7,...,0.95,0.0,0.0,0.0,55.2,0.0,0.0,0.0,55.2,1324
2053-09-22,2053,9,22,2052,393,0.0,1,48,8,7,...,0.94,0.0,0.0,0.0,54.1,0.0,0.0,0.0,54.1,1299
2053-09-23,2053,9,23,2052,391,0.0,1,48,8,7,...,0.94,0.0,0.0,0.0,53.6,0.0,0.0,0.0,53.6,1287
2053-09-24,2053,9,24,2052,391,0.0,1,48,8,7,...,0.94,0.0,0.0,0.0,53.6,0.0,0.0,0.0,53.6,1287
2053-09-25,2053,9,25,2052,391,0.0,1,48,8,7,...,0.94,0.0,0.0,0.0,53.6,0.0,0.0,0.0,53.6,1287
2053-09-26,2053,9,26,2052,387,0.0,1,48,8,7,...,0.94,0.0,0.0,0.0,52.8,0.0,0.0,0.0,52.8,1268
2053-09-27,2053,9,27,2052,383,0.0,1,48,8,7,...,0.94,0.0,0.0,0.0,52.0,0.0,0.0,0.0,52.0,1248


## Weekly & Weekly Slim
Slim has less columns and values are rounded so file size is reduced

In [130]:
if not 'WaterWeek' in daily.columns:
    daily['DateVal']=daily.index
    daily['WaterDay']=daily.apply(lambda x: (x['DateVal']-pd.Timestamp(x['WaterYear'], 10, 1)).days+1,axis=1)
    daily['WaterWeek']=np.floor((daily['WaterDay']-1)/7)+1

In [132]:
weekly=daily.groupby(["WaterYear","WaterWeek"]).mean().drop(['Month','WaterMonth','Year','MonthId','Day','WaterDay','EWRBandNo','EWRRefExceedance','Turbines','Energy'],axis=1)

if 'VicFalls' in weekly.columns:
    weekly=weekly.drop(['VicFalls','Conversion','Volume'],axis=1)

weekly=weekly.join(daily.groupby(["WaterYear","WaterWeek"]).agg(    
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
))
weekly=weekly.join(daily[["WaterYear","WaterWeek","Flow"]].groupby(["WaterYear","WaterWeek"]).count().rename(columns={'Flow':'Count'}))
weekly['CapFactor']=weekly['Energy']/(180*24*weekly['Count'])
weekly=weekly.join(daily.reset_index('Date')[["WaterYear","WaterWeek","Date"]].groupby(["WaterYear","WaterWeek"]).first().rename(columns={'Date':'StartDate'}))

weekly.tail(12)

CC_Factor  SampleFlow  Impact_2070        Flow  \
WaterYear WaterWeek                                                   
2052      42.0          0.7875  982.334815    -0.170000  850.824741   
          43.0          0.7875  875.539247    -0.170000  758.326430   
          44.0          0.7875  780.130277    -0.170000  675.690336   
          45.0          0.7875  701.716222    -0.170000  607.773963   
          46.0          0.7875  637.286855    -0.170000  551.970077   
          47.0          0.7875  582.198166    -0.170000  504.256387   
          48.0          0.7875  546.466198    -0.165714  475.105288   
          49.0          0.7875  509.755748    -0.140000  453.555177   
          50.0          0.7875  475.036875    -0.140000  422.664060   
          51.0          0.7875  453.653102    -0.140000  403.637848   
          52.0          0.7875  434.877823    -0.140000  386.932543   
          53.0          0.7875  426.632881    -0.140000  379.596606   

                     EWRChannelA  EWRChannelC  EWRChannelD  EWRChannelE  \
WaterYear WaterWeek                                                       
2052      42.0         63.000000    12.000000    13.000000    13.000000   
          43.0         63.000000    12.000000    13.000000    13.000000   
          44.0         60.142857    10.857143    11.285714     9.571429   
          45.0         58.000000    10.000000    10.000000     7.000000   
          46.0         58.000000    10.000000    10.000000     7.000000   
          47.0         58.000000    10.000000    10.000000     7.000000   
          48.0         56.571429     9.714286     9.571429     6.571429   
          49.0         48.000000     8.000000     7.000000     4.000000   
          50.0         48.000000     8.000000     7.000000     4.000000   
          51.0         48.000000     8.000000     7.000000     4.000000   
          52.0         48.000000     8.000000     7.000000     4.000000   
          53.0         48.000000     8.000000     7.000000     4.000000   

                     EWRChannelFG    EWRTotal  ...  HeadTurbine1_max  \
WaterYear WaterWeek                            ...                     
2052      42.0         155.000000  256.000000  ...         22.294168   
          43.0         155.000000  256.000000  ...         22.630195   
          44.0         133.285714  225.142857  ...         23.062597   
          45.0         117.000000  202.000000  ...         23.384058   
          46.0         117.000000  202.000000  ...         23.781546   
          47.0         117.000000  202.000000  ...         24.022185   
          48.0         111.571429  194.000000  ...         24.082936   
          49.0          79.000000  146.000000  ...         24.248243   
          50.0          79.000000  146.000000  ...         24.310360   
          51.0          79.000000  146.000000  ...         24.157974   
          52.0          79.000000  146.000000  ...         24.273305   
          53.0          79.000000  146.000000  ...         24.273305   

                     HeadTurbine1_min  LevelTailwater_max  LevelTailwater_min  \
WaterYear WaterWeek                                                             
2052      42.0              21.843110          967.384934          967.074440   
          43.0              22.088793          967.014627          966.682254   
          44.0              22.729092          966.618689          966.323058   
          45.0              23.096381          966.300929          966.103337   
          46.0              23.469014          966.043690          965.820252   
          47.0              23.781546          965.820252          965.643918   
          48.0              23.702511          965.643918          965.548440   
          49.0              24.046738          965.563563          965.416766   
          50.0              23.935792          965.404091          965.332602   
          51.0              24.011515          965.315368          965.2420

In [133]:
weekly_slim=weekly.loc[:,
                     [
                      'StartDate',
                      'Flow',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion':2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,
                    })

## Calendar Months

In [134]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly=calmonthly.join(monthly.groupby(["WaterMonth"]).agg(    
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_mean=('EWRProportion', 'mean'),
   EWRProportion_median=('EWRProportion', 'median'),
   EWRProportion_min=('EWRProportion', 'min'),
    
    EWRTotal_max=('EWRTotal', 'max'),
   EWRTotal_mean=('EWRTotal', 'mean'),
   EWRTotal_median=('EWRTotal', 'median'),
   EWRTotal_min=('EWRTotal', 'min'),   
    
   FlowCanal_max=('FlowCanal', 'max'),
   FlowCanal_mean=('FlowCanal', 'mean'),
   FlowCanal_median=('FlowCanal', 'median'),
   FlowCanal_min=('FlowCanal', 'min'),     
    
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_mean=('HeadTurbine1', 'mean'),
   HeadTurbine1_median=('HeadTurbine1', 'median'),
   HeadTurbine1_min=('HeadTurbine1', 'min'), 

   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_mean=('LevelTailwater', 'mean'),    
   LevelTailwater_median=('LevelTailwater', 'median'),    
   LevelTailwater_min=('LevelTailwater', 'min'),  
   LevelHeadpond_max=('LevelHeadpond', 'max'),
   LevelHeadpond_mean=('LevelHeadpond', 'mean'),    
   LevelHeadpond_median=('LevelHeadpond', 'median'),    
   LevelHeadpond_min=('LevelHeadpond', 'min'),  
   Turbines_mean=('Turbines', 'mean'),      
   Power_max=('Power', 'max'),
   Power_mean=('Power', 'mean'),
   Power_median=('Power', 'median'),
   Power_min=('Power', 'min'),    
   Energy_max=('Energy', 'max'),
   Energy_mean=('Energy', 'mean'),
   Energy_median=('Energy', 'median'),    
   Energy_min=('Energy', 'min'),
))
calmonthly['Energy_P95']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.05)
calmonthly['Energy_P90']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.10)
calmonthly['Energy_P75']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.25)
calmonthly['Energy_P25']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.75)
calmonthly['Energy_P10']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.90)
calmonthly['Energy_P5']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.95)

calmonthly['Energy_stdev']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').std()
calmonthly['Energy_coefvar']=calmonthly['Energy_stdev']/calmonthly['Energy_mean']

calmonthly

,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,Energy_median,Energy_min,Energy_P95,Energy_P90,Energy_P75,Energy_P25,Energy_P10,Energy_P5,Energy_stdev,Energy_coefvar
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1,Oct,10,0.496064,0.413267,0.413948,0.325567,120.0,93.655018,97.000000,52.0,...,20166.272357,15447.754616,16795.057008,17417.140777,18339.172688,24130.916010,25867.124384,29386.451978,4147.278641,0.194338
2,Nov,11,0.418602,0.349914,0.358188,0.257784,114.0,89.472222,95.166667,50.0,...,25054.524838,18984.929541,20234.055485,20778.164976,22530.852525,29854.078242,32686.184076,35631.114958,5604.737089,0.211547
3,Dec,12,0.293462,0.249871,0.254411,0.179589,126.0,100.472222,104.000000,59.0,...,44499.097273,37384.633949,39437.883881,40337.532837,42325.514914,54456.155534,62993.836520,75105.500308,11646.520007,0.233986
4,Jan,1,0.233585,0.185773,0.191414,0.108941,150.0,118.277778,126.129032,73.0,...,73655.215566,52377.311949,57071.703802,63363.111936,67302.109975,85630.951997,105580.140566,126620.932987,19497.008329,0.244644
5,Feb,2,0.179753,0.131039,0.141110,0.026742,197.0,139.322660,137.000000,85.0,...,95764.137014,7712.678171,65160.467742,76025.441545,89057.593320,107971.302532,114734.614832,116996.306206,20848.454953,0.220605
6,Mar,3,0.196364,0.098700,0.097701,0.054868,315.0,195.833333,192.935484,107.0,...,109773.933627,29633.185228,61611.374791,74610.206408,99742.904282,119637.320635,128508.326486,130575.433613,24167.682898,0.231584
7,Apr,4,0.172183,0.084941,0.081772,0.064789,364.0,233.527778,243.000000,125.0,...,102175.664328,25863.853941,77788.998221,79490.695169,88373.589627,116915.479886,122338.028500,127369.758536,20288.743846,0.201782
8,May,5,0.246596,0.141301,0.141539,0.078109,340.0,266.194444,290.516129,147.0,...,125183.397670,71792.316707,96914.410162,105957.555686,114423.516325,130055.648096,133427.272755,135195.798364,14767.229007,0.122369
9,Jun,6,0.335802,0.219471,0.218060,0.127919,293.0,213.944444,210.000000,119.0,...,106762.058518,37481.399954,67590.404880,80754.118725,84938.966101,119212.303098,123807.502971,124110.006222,20976.709332,0.207747


## Flow duration curve

In [138]:
if file_infix=='':
    daily['ExceedanceRnd']=np.round(daily['Exceedance'],2)
    fdc=daily.reset_index().groupby('ExceedanceRnd').mean().drop(['VicFalls','Conversion','Year','Month','Day','MonthId','WaterYear','WaterDay','WaterMonth','WaterWeek'],axis=1)
    fdc

## Add overall summary data to the model file

In [137]:
if file_infix=='':
    models.at[model_name,'Mean']=yearly['Energy'].mean()
    models.at[model_name,'Max']=yearly['Energy'].max()
    models.at[model_name,'Min']=yearly['Energy'].min()

    models.at[model_name,'P95']=yearly['Energy'].quantile(0.05)
    models.at[model_name,'P90']=yearly['Energy'].quantile(0.10)
    models.at[model_name,'P75']=yearly['Energy'].quantile(0.25)
    models.at[model_name,'P50']=yearly['Energy'].quantile(0.5)
    models.at[model_name,'P25']=yearly['Energy'].quantile(0.75)
    models.at[model_name,'P10']=yearly['Energy'].quantile(0.90)
    models.at[model_name,'P5']=yearly['Energy'].quantile(0.95)

    models.at[model_name,'StDev']=yearly['Energy'].std()
    models.at[model_name,'CoefVar']=models.at[model_name,'StDev']/models.at[model_name,'Mean']

    models.at[model_name,'CapFactor']=yearly['CapFactor'].mean()

    models.at[model_name,'CalYearMean']=calyearly['Energy'].mean()
    models.at[model_name,'CalYearStDev']=calyearly['Energy'].std()
    models.at[model_name,'CalYearCoefVar']=models.at[model_name,'CalYearStDev']/models.at[model_name,'CalYearMean']

    models.at[model_name,'Power_max']=yearly['Power_max'].max()
    models.at[model_name,'CapFactor_Power_max']=calyearly['Energy'].mean()/(yearly['Power_max'].max()*24*365)

    models.loc[model_name]



In [ ]:
if file_infix=='':
    autocorrel=pd.DataFrame(statools.acf(yearly['Energy'],fft=True,qstat=False,alpha=0.05))
    autocorrel

## Save Files

In [142]:
if file_infix=='':
    outp=output_data+folder+file_infix+out_prefix
else:
    outp=output_data+folder+file_infix[1:]+'_'+out_prefix
if dryrun==False:
    daily.to_csv(outp+'_pe_daily.csv')
    monthly.to_csv(outp+'_pe_monthly.csv')
    yearly.to_csv(outp+'_pe_yearly.csv')
    edc.to_csv(outp+'_pe_percentiles.csv')
    monthly_day.to_csv(outp+'_pe_monthly_day.csv')
    daily_slim.to_csv(outp+'_pe_daily_slim.csv')
    weekly.to_csv(outp+'_pe_weekly.csv')
    weekly_slim.to_csv(outp+'_pe_weekly_slim.csv')
    calmonthly.to_csv(outp+'_pe_calmonthly.csv')
    power_exceed.to_csv(outp+'_pe_power_exceed.csv')
    if file_infix=='':
        fdc.to_csv(outp+'_pe_fdc.csv')
        autocorrel.to_csv(outp+'_pe_autocorrel.csv')
        models.to_csv(input_data + "models.csv")

In [141]:
file_infix[1:]

'WC1_1'